In [ ]:
#|default_exp distributed.models.xgb

# XGBForecast

> XGBoost forecaster

Wrapper of `xgboost.dask.DaskXGBRegressor` that adds a `model_` property that contains the fitted model and is sent to the workers in the forecasting step.

In [ ]:
#|export
import xgboost as xgb

In [ ]:
#|export
class XGBForecast(xgb.dask.DaskXGBRegressor):
    @property
    def model_(self):
        local = xgb.XGBRegressor(**self.get_params())
        local._Booster = self._Booster
        return local

In [ ]:
import dask
import dask.dataframe as dd
import numpy as np
import pandas as pd

from dask.distributed import Client
from mlforecast.distributed import DistributedForecast
from mlforecast.utils import generate_daily_series

In [ ]:
client = Client(n_workers=2, threads_per_worker=1)

In [ ]:
series = generate_daily_series(100)
distr_series = dd.from_pandas(series, npartitions=2)
fcst = DistributedForecast(
    XGBForecast(random_state=0),
    freq='D',
    lags=[7],
)
fcst.fit(distr_series)
actual = fcst.predict(1).compute()

def get_updates(ts):
    ts._predict_setup()
    upd = ts._update_features()
    return upd.drop(columns='ds')

upd_futures = client.map(get_updates, fcst.dts.ts)
upd_ddf = dd.from_delayed(upd_futures)
expected = fcst.models_[0].predict(upd_ddf).compute()

np.testing.assert_equal(actual['XGBRegressor'].values, expected)

[21:06:28] task [xgboost.dask]:tcp://127.0.0.1:40603 got new rank 0
[21:06:28] task [xgboost.dask]:tcp://127.0.0.1:39039 got new rank 1


In [ ]:
client.close()